In [ ]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-1962, -365, 93)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if i == 0:
                allstock = stockdata
                # print(allstock)
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            pass
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            allstock.loc[id, 'RS 50rate>80'] = (allstock.loc[id, 'RS 50rate'] > 80)
            allstock.loc[id, 'RS EMA250rate>80'] = (allstock.loc[id, 'RS EMA250rate']>80)
            allstock.loc[id, 'RS EMA50rate>80'] = (allstock.loc[id, 'RS EMA50rate'] > 80)
            allstock.loc[id, 'RS EMA250rate>80 and EMA50rate>80'] = all(allstock.loc[id, ['RS EMA250rate>80','RS EMA50rate>80']])
            allstock.loc[id, 'RS 20rate>70'] = (allstock.loc[id, 'RS 20rate'] > 70)
            allstock.loc[id, 'RS 250rate>70'] = (allstock.loc[id, 'RS 250rate'] > 70)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 100*1000)
            allstock.loc[id, 'Mark Tamplate'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 第一個日期選出符合條件的股票並匯出excel
    # print(allstock['Mark Tamplate'])
    allstock.dropna(subset='Mark Tamplate', inplace=True)
    apexstock = allstock.loc[allstock['RS EMA250rate>80 and EMA50rate>80'], ['產業別', 'RS 250rate', 'RS 50rate', 'RS 20rate', 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA',
                                                        'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate']].astype(float).round(1)
    apexstock = apexstock.dropna()
    print(len(allstock.loc[allstock['RS EMA250rate>80 and EMA50rate>80']].index))
    print(str(day))
    # apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/history_choosen/' + str(day).split(' ')[0] + 'RS選股(EMA)' + '.xlsx', encoding='utf-8-sig')


    IDs = apexstock.index.values
    day2 = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+365))) , '%Y-%m-%d' )
    for i, ID in enumerate(IDs):
        try:
            stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
        except:
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
            except Exception as e:
                print(ID)
                print(e)
        if i == 0:
            rocp_df = stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP']
            # rocp_df = rocp_df.loc[(rocp_df.index<day2)]
        else:
            rocp = pd.Series(stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP'], name = str(ID))
            rocp_df = pd.concat([rocp, rocp_df], axis=1)

    roc_df = rocp_df/100
    roc_df = roc_df+1   
    season1_profit = abs(roc_df.iloc[:63]).prod()
    season2_profit = abs(roc_df.iloc[63:126]).prod()
    season3_profit = abs(roc_df.iloc[126:189]).prod()
    season4_profit = abs(roc_df.iloc[189:]).prod()
    all_profit = abs(roc_df.iloc[:]).prod()
    season1_profit = season1_profit-1
    season2_profit = season2_profit-1
    season3_profit = season3_profit-1
    season4_profit = season4_profit-1
    all_profit = all_profit-1
    season1_profit = season1_profit.sum()
    season2_profit = season2_profit.sum()
    season3_profit = season3_profit.sum()
    season4_profit = season4_profit.sum()
    all_profit = all_profit.sum()
    profit = {'season1_profit':season1_profit,
            'season2_profit':season2_profit,
            'season3_profit':season3_profit,
            'season4_profit':season4_profit,
            'all_profit':all_profit,
            'number of stock':len(allstock.loc[allstock['RS EMA250rate>80 and EMA50rate>80']].index)}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([profit_df, all_df])
    display(all_df)
    # print(season1_profit.sum())
    # print(season2_profit.sum())
    # print(season3_profit.sum())
    # print(season4_profit.sum())
    # print(all_profit.sum())

In [3]:
avg_profit_df = pd.DataFrame([], columns=all_df.columns)
for ind in all_df.index.values:
    avg_profit_df.loc[ind] = 100*all_df.loc[ind]/all_df.loc[ind, 'number of stock']
avg_profit_df.loc['total'] = avg_profit_df.sum()
display(avg_profit_df)

,season1_profit,season2_profit,season3_profit,season4_profit,all_profit,number of stock
2022-07-18 00:00:00,-4.787371,8.352847,20.920284,10.835198,41.079318,100.0
2022-04-15 00:00:00,-14.523439,-1.600812,9.150844,13.779371,5.657751,100.0
2022-01-12 00:00:00,-7.174702,-3.391107,-11.728193,10.598042,-13.734550,100.0
2021-10-12 00:00:00,7.945111,-0.138767,-8.337389,-5.887971,-7.173620,100.0
2021-07-12 00:00:00,-14.734726,14.044098,-3.559689,-18.810484,-24.815915,100.0
2021-04-08 00:00:00,9.940622,-9.004485,16.731986,-7.120242,1.905947,100.0
2021-01-05 00:00:00,29.286341,12.448454,-13.132600,18.044766,46.003345,100.0
2020-10-05 00:00:00,16.754733,22.334004,6.116328,-11.821410,34.191276,100.0
2020-07-03 00:00:00,0.834314,9.328375,15.904602,0.547248,31.127309,100.0
2020-04-01 00:00:00,37.383158,2.270827,8.768582,12.323492,71.990952,100.0


In [ ]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-1962, -365, 93)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if i == 0:
                allstock = stockdata
                # print(allstock)
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            pass
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            allstock.loc[id, 'RS 50rate>80'] = (allstock.loc[id, 'RS 50rate'] > 80)
            allstock.loc[id, 'RS EMA250rate>85'] = (allstock.loc[id, 'RS EMA250rate']>85)
            allstock.loc[id, 'RS EMA50rate>80'] = (allstock.loc[id, 'RS EMA50rate'] > 80)
            allstock.loc[id, 'RS 20rate>70'] = (allstock.loc[id, 'RS 20rate'] > 70)
            allstock.loc[id, 'RS 250rate>70'] = (allstock.loc[id, 'RS 250rate'] > 70)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 100*1000)
            allstock.loc[id, 'new_template'] = all(allstock.loc[id, ['year low sort', 'year high sort', 'RS EMA50rate>80', 'Volume 50MA>100k']])
            allstock.loc[id, 'Mark Tamplate'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 第一個日期選出符合條件的股票並匯出excel
    # print(allstock['Mark Tamplate'])
    allstock.dropna(subset='Mark Tamplate', inplace=True)
    apexstock = allstock.loc[allstock['new_template'], ['產業別', 'RS 250rate', 'RS 50rate', 'RS 20rate', 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA',
                                                        'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate']].astype(float).round(1)
    apexstock = apexstock.dropna()
    print(len(allstock.loc[allstock['new_template']].index))
    print(str(day))
    apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/history_choosen/' + str(day).split(' ')[0] + 'RS選股(EMA)' + '.xlsx', encoding='utf-8-sig')


    IDs = apexstock.index.values
    day2 = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+365))) , '%Y-%m-%d' )
    for i, ID in enumerate(IDs):
        try:
            stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
        except:
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
            except Exception as e:
                print(ID)
                print(e)
        if i == 0:
            rocp_df = stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP']
            # rocp_df = rocp_df.loc[(rocp_df.index<day2)]
        else:
            rocp = pd.Series(stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP'], name = str(ID))
            rocp_df = pd.concat([rocp, rocp_df], axis=1)

    roc_df = rocp_df/100
    roc_df = roc_df+1   
    season1_profit = abs(roc_df.iloc[:63]).prod()
    season2_profit = abs(roc_df.iloc[63:126]).prod()
    season3_profit = abs(roc_df.iloc[126:189]).prod()
    season4_profit = abs(roc_df.iloc[189:]).prod()
    all_profit = abs(roc_df.iloc[:]).prod()
    season1_profit = season1_profit-1
    season2_profit = season2_profit-1
    season3_profit = season3_profit-1
    season4_profit = season4_profit-1
    all_profit = all_profit-1
    season1_profit = season1_profit.sum()
    season2_profit = season2_profit.sum()
    season3_profit = season3_profit.sum()
    season4_profit = season4_profit.sum()
    all_profit = all_profit.sum()
    profit = {'season1_profit':season1_profit,
            'season2_profit':season2_profit,
            'season3_profit':season3_profit,
            'season4_profit':season4_profit,
            'all_profit':all_profit,
            'number of stock':len(allstock.loc[allstock['new_template']].index)}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([profit_df, all_df])
    display(all_df)
    # print(season1_profit.sum())
    # print(season2_profit.sum())
    # print(season3_profit.sum())
    # print(season4_profit.sum())
    # print(all_profit.sum())

In [5]:
avg_profit_df2 = pd.DataFrame([], columns=all_df.columns)
for ind in all_df.index.values:
    avg_profit_df2.loc[ind] = 100*all_df.loc[ind]/all_df.loc[ind, 'number of stock']
avg_profit_df2.loc['total'] = avg_profit_df2.sum()
display(avg_profit_df2)

,season1_profit,season2_profit,season3_profit,season4_profit,all_profit,number of stock
2022-07-18 00:00:00,-4.619967,8.553092,19.733815,14.619597,46.402803,100.0
2022-04-15 00:00:00,-11.418364,-1.888099,8.196691,13.776977,8.701100,100.0
2022-01-12 00:00:00,-7.374013,-2.992649,-9.145952,9.542025,-11.506570,100.0
2021-10-12 00:00:00,8.106414,-0.589839,-6.236752,-5.520927,-4.873029,100.0
2021-07-12 00:00:00,-10.908153,16.708797,-4.374704,-17.188932,-18.373447,100.0
2021-04-08 00:00:00,8.095834,-7.711744,18.311426,-7.162837,3.516973,100.0
2021-01-05 00:00:00,26.348142,13.374397,-12.208541,18.307524,45.041085,100.0
2020-10-05 00:00:00,20.133862,22.065101,6.134891,-11.527101,36.670114,100.0
2020-07-03 00:00:00,1.812963,11.639301,16.924493,-0.636743,33.891024,100.0
2020-04-01 00:00:00,31.007522,1.115753,7.227269,12.888877,70.094995,100.0


In [ ]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-1962, -365, 62)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if i == 0:
                allstock = stockdata
                # print(allstock)
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            pass
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            allstock.loc[id, 'RS 50rate>80'] = (allstock.loc[id, 'RS 50rate'] > 80)
            allstock.loc[id, 'RS EMA250rate>85'] = (allstock.loc[id, 'RS EMA250rate']>85)
            allstock.loc[id, 'RS EMA50rate>80'] = (allstock.loc[id, 'RS EMA50rate'] > 80)
            allstock.loc[id, 'RS 20rate>70'] = (allstock.loc[id, 'RS 20rate'] > 70)
            allstock.loc[id, 'RS 250rate>70'] = (allstock.loc[id, 'RS 250rate'] > 70)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 100*1000)
            allstock.loc[id, 'new_template'] = all(allstock.loc[id, ['year low sort', 'year high sort', 'RS EMA50rate>80', 'Volume 50MA>100k']])
            allstock.loc[id, 'Mark Tamplate'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 第一個日期選出符合條件的股票並匯出excel
    # print(allstock['Mark Tamplate'])
    allstock.dropna(subset='Mark Tamplate', inplace=True)
    apexstock = allstock.loc[allstock['new_template'], ['產業別', 'RS 250rate', 'RS 50rate', 'RS 20rate', 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA',
                                                        'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate']].astype(float).round(1)
    apexstock = apexstock.dropna()
    print(len(allstock.loc[allstock['new_template']].index))
    print(str(day))
    apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/history_choosen/' + str(day).split(' ')[0] + 'RS選股(EMA)' + '.xlsx', encoding='utf-8-sig')


    IDs = apexstock.index.values
    day2 = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+365))) , '%Y-%m-%d' )
    for i, ID in enumerate(IDs):
        try:
            stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
        except:
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
            except Exception as e:
                print(ID)
                print(e)
        if i == 0:
            rocp_df = stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP']
            # rocp_df = rocp_df.loc[(rocp_df.index<day2)]
        else:
            rocp = pd.Series(stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP'], name = str(ID))
            rocp_df = pd.concat([rocp, rocp_df], axis=1)

    roc_df = rocp_df/100
    roc_df = roc_df+1   
    season1_profit = abs(roc_df.iloc[:42]).prod()
    season2_profit = abs(roc_df.iloc[42:84]).prod()
    season3_profit = abs(roc_df.iloc[84:126]).prod()
    season4_profit = abs(roc_df.iloc[126:168]).prod()
    season5_profit = abs(roc_df.iloc[168:210]).prod()
    season6_profit = abs(roc_df.iloc[210:]).prod()
    all_profit = abs(roc_df.iloc[:]).prod()
    season1_profit = season1_profit-1
    season2_profit = season2_profit-1
    season3_profit = season3_profit-1
    season4_profit = season4_profit-1
    season5_profit = season5_profit-1
    season6_profit = season6_profit-1
    all_profit = all_profit-1
    season1_profit = season1_profit.sum()
    season2_profit = season2_profit.sum()
    season3_profit = season3_profit.sum()
    season4_profit = season4_profit.sum()
    season5_profit = season5_profit.sum()
    season6_profit = season6_profit.sum()
    all_profit = all_profit.sum()
    profit = {'season1_profit':season1_profit,
            'season2_profit':season2_profit,
            'season3_profit':season3_profit,
            'season4_profit':season4_profit,
            'season5_profit':season3_profit,
            'season6_profit':season4_profit,
            'all_profit':all_profit,
            'number of stock':len(allstock.loc[allstock['new_template']].index)}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([profit_df, all_df])
    display(all_df)
    # print(season1_profit.sum())
    # print(season2_profit.sum())
    # print(season3_profit.sum())
    # print(season4_profit.sum())
    # print(all_profit.sum())

In [8]:
avg_profit_df3 = pd.DataFrame([], columns=all_df.columns)
for ind in all_df.index.values:
    avg_profit_df3.loc[ind] = 100*all_df.loc[ind]/all_df.loc[ind, 'number of stock']
avg_profit_df3.loc['total'] = avg_profit_df3.sum()
display(avg_profit_df3)

,season1_profit,season2_profit,season3_profit,season4_profit,season5_profit,season6_profit,all_profit,number of stock
2022-06-17 00:00:00,6.290105,-10.160634,13.179240,9.832575,13.179240,9.832575,33.005397,100.0
2022-04-18 00:00:00,-6.974363,2.648379,-9.889285,12.078980,-9.889285,12.078980,10.937622,100.0
2022-02-14 00:00:00,-1.465426,-1.623560,2.050952,-11.430896,2.050952,-11.430896,-1.593421,100.0
2021-12-13 00:00:00,-5.076533,-10.013064,-1.846098,7.446959,-1.846098,7.446959,-13.762382,100.0
2021-10-12 00:00:00,9.568500,-1.805878,-0.210356,-7.251484,-0.210356,-7.251484,-4.873029,100.0
2021-08-11 00:00:00,-2.315473,15.155667,0.866164,-3.273100,0.866164,-3.273100,0.589011,100.0
2021-06-10 00:00:00,11.149598,-5.931783,14.357298,-1.649982,14.357298,-1.649982,12.273447,100.0
2021-04-09 00:00:00,-5.913316,12.161650,-8.212459,16.917400,-8.212459,16.917400,5.567847,100.0
2021-02-17 00:00:00,20.860022,-0.714077,-2.524867,2.501809,-2.524867,2.501809,31.240338,100.0
2020-12-07 00:00:00,3.407475,35.815399,-5.698860,2.793165,-5.698860,2.793165,52.268566,100.0


In [ ]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-5000, -30, 62)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if i == 0:
                allstock = stockdata
                # print(allstock)
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            pass
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            allstock.loc[id, 'RS 50rate>80'] = (allstock.loc[id, 'RS 50rate'] > 80)
            allstock.loc[id, 'RS EMA250rate>85'] = (allstock.loc[id, 'RS EMA250rate']>85)
            allstock.loc[id, 'RS EMA50rate>80'] = (allstock.loc[id, 'RS EMA50rate'] > 80)
            allstock.loc[id, 'RS 20rate>70'] = (allstock.loc[id, 'RS 20rate'] > 70)
            allstock.loc[id, 'RS 250rate>70'] = (allstock.loc[id, 'RS 250rate'] > 70)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 100*1000)
            allstock.loc[id, 'new_template'] = all(allstock.loc[id, ['year low sort', 'year high sort', 'RS EMA50rate>80', 'Volume 50MA>100k']])
            allstock.loc[id, 'Mark Tamplate'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 第一個日期選出符合條件的股票並匯出excel
    # print(allstock['Mark Tamplate'])
    allstock.dropna(subset='Mark Tamplate', inplace=True)
    apexstock = allstock.loc[allstock['new_template'], ['產業別', 'RS 250rate', 'RS 50rate', 'RS 20rate', 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA',
                                                        'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate']].astype(float).round(1)
    apexstock = apexstock.dropna()
    print(len(allstock.loc[allstock['new_template']].index))
    print(str(day))
    apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/history_choosen/' + str(day).split(' ')[0] + 'RS選股(EMA)' + '.xlsx', encoding='utf-8-sig')


    IDs = apexstock.index.values
    day2 = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+365))) , '%Y-%m-%d' )
    for i, ID in enumerate(IDs):
        try:
            stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
        except:
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
            except Exception as e:
                print(ID)
                print(e)
        if i == 0:
            rocp_df = stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP']
            # rocp_df = rocp_df.loc[(rocp_df.index<day2)]
        else:
            rocp = pd.Series(stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP'], name = str(ID))
            rocp_df = pd.concat([rocp, rocp_df], axis=1)

    roc_df = rocp_df/100
    roc_df = roc_df+1   
    season1_profit = abs(roc_df.iloc[:42]).prod()
    season2_profit = abs(roc_df.iloc[42:84]).prod()
    season3_profit = abs(roc_df.iloc[84:126]).prod()
    season4_profit = abs(roc_df.iloc[126:168]).prod()
    season5_profit = abs(roc_df.iloc[168:210]).prod()
    season6_profit = abs(roc_df.iloc[210:]).prod()
    all_profit = abs(roc_df.iloc[:]).prod()
    season1_profit = season1_profit-1
    season2_profit = season2_profit-1
    season3_profit = season3_profit-1
    season4_profit = season4_profit-1
    season5_profit = season5_profit-1
    season6_profit = season6_profit-1
    all_profit = all_profit-1
    season1_profit = season1_profit.sum()
    season2_profit = season2_profit.sum()
    season3_profit = season3_profit.sum()
    season4_profit = season4_profit.sum()
    season5_profit = season5_profit.sum()
    season6_profit = season6_profit.sum()
    all_profit = all_profit.sum()
    profit = {'season1_profit':season1_profit,
            'season2_profit':season2_profit,
            'season3_profit':season3_profit,
            'season4_profit':season4_profit,
            'season5_profit':season3_profit,
            'season6_profit':season4_profit,
            'all_profit':all_profit,
            'number of stock':len(allstock.loc[allstock['new_template']].index)}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([profit_df, all_df])
    display(all_df)
    # print(season1_profit.sum())
    # print(season2_profit.sum())
    # print(season3_profit.sum())
    # print(season4_profit.sum())
    # print(all_profit.sum())

In [10]:
avg_profit_df4 = pd.DataFrame([], columns=all_df.columns)
for ind in all_df.index.values:
    avg_profit_df4.loc[ind] = 100*all_df.loc[ind]/all_df.loc[ind, 'number of stock']
avg_profit_df4.loc['total'] = avg_profit_df4.sum()
display(avg_profit_df4)

,season1_profit,season2_profit,season3_profit,season4_profit,season5_profit,season6_profit,all_profit,number of stock
2023-06-26 00:00:00,-1.555372,0.000000,0.000000,0.000000,0.000000,0.000000,-1.555372,100.0
2023-04-24 00:00:00,11.325742,-1.175862,0.000000,0.000000,0.000000,0.000000,9.956070,100.0
2023-02-20 00:00:00,9.034020,12.655029,-2.680092,0.000000,-2.680092,0.000000,19.262513,100.0
2022-12-20 00:00:00,15.847666,8.932842,8.248062,-3.565431,8.248062,-3.565431,34.274122,100.0
2022-10-19 00:00:00,7.778650,9.082965,6.455351,8.329397,6.455351,8.329397,30.912822,100.0
...,...,...,...,...,...,...,...,...
2010-05-31 00:00:00,6.996634,5.198494,-4.942936,7.252543,-4.942936,7.252543,7.605392,100.0
2010-03-29 00:00:00,-9.219407,10.099170,2.599910,1.771843,2.599910,1.771843,7.333098,100.0
2010-01-25 00:00:00,5.415167,-7.844399,8.125017,-1.181300,8.125017,-1.181300,12.896339,100.0
2009-11-24 00:00:00,11.385224,3.719616,-3.625061,9.344471,-3.625061,9.344471,12.490683,100.0


In [ ]:
avg_profit_df4.to_excel(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\ETF test\year low sort, year high sort, RS EMA50rate大於80, Volume 50MA大於100k.xlsx')

In [ ]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-5002, -30, 62)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    f = 1
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if f == 1:
                allstock = stockdata
                f = 0
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            # print(e)
            pass
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            allstock.loc[id, 'RS 50rate>80'] = (allstock.loc[id, 'RS 50rate'] > 80)
            allstock.loc[id, 'RS EMA250rate>80'] = (allstock.loc[id, 'RS EMA250rate']>80)
            allstock.loc[id, 'RS EMA20rate>80'] = (allstock.loc[id, 'RS EMA20rate'] > 80)
            allstock.loc[id, 'RS EMA250rate>70'] = (allstock.loc[id, 'RS EMA250rate']>70)
            allstock.loc[id, 'RS EMA250rate<90'] = (allstock.loc[id, 'RS EMA250rate']<90)
            allstock.loc[id, 'RS EMA20rate>75'] = (allstock.loc[id, 'RS EMA20rate'] > 75)
            allstock.loc[id, 'ATR250/price'] = (allstock.loc[id, 'ATR250'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR50/price'] = (allstock.loc[id, 'ATR50'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR20/price'] = (allstock.loc[id, 'ATR20'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR250/price<0.03'] = (allstock.loc[id, 'ATR250/price'] < 0.04)
            allstock.loc[id, 'ATR50/price<0.03'] = (allstock.loc[id, 'ATR50/price'] < 0.05)
            allstock.loc[id, 'ATR20/price<0.03'] = (allstock.loc[id, 'ATR20/price'] < 0.04)            
            allstock.loc[id, 'RS 20rate>70'] = (allstock.loc[id, 'RS 20rate'] > 70)
            allstock.loc[id, 'RS 250rate>70'] = (allstock.loc[id, 'RS 250rate'] > 70)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 250*1000)
            allstock.loc[id, 'Mark Tamplate'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
            allstock.loc[id, 'T1'] = all(allstock.loc[id, ['RS EMA250rate>80','RS EMA20rate>80', '200MA trending up 60d', 'Volume 50MA>100k']])
            allstock.loc[id, 'T2'] = all(allstock.loc[id, ['RS EMA250rate>80','RS EMA20rate>80', '200MA trending up 60d', 'ATR50/price<0.03', 'Volume 50MA>100k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 第一個日期選出符合條件的股票並匯出excel
    # print(allstock['Mark Tamplate'])
    allstock.dropna(subset='Mark Tamplate', inplace=True)
    template1_df = allstock.loc[allstock['T1'], ['產業別', 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                                                             , 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA',
                                                        'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    template2_df = allstock.loc[allstock['T2'], ['產業別', 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                                                             , 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA',
                                                        'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    template1_df.dropna(inplace=True)
    template2_df.dropna(inplace=True)
    apexstock = template2_df.sort_values(by='RS EMA50rate', ascending=True).iloc[-60:]
    template1_df = template1_df.sort_values(by='RS EMA50rate', ascending=True).iloc[-60:]
    template2_num = int(len(template1_df.index.values)/4)
    template2_df = template2_df.sort_values(by='RS EMA50rate', ascending=True).iloc[-template2_num:]
    print(f'T2 len:{len(template2_df.index.values)}')
    template2_df = template2_df.loc[~template2_df.index.isin(template1_df.index)]
    
    # apexstock = pd.concat([template1_df, template2_df])
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']].astype(float).round(1)
    apexstock = apexstock.dropna()
    print(f'T1 len:{len(template1_df.index.values)}')
    print(f'T2 len:{len(template2_df.index.values)}')
    print(f'apexstock len:{len(apexstock.index.values)}')
    print(str(day))
    # apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/ETF test/stock choose/' + str(day).split(' ')[0] + 'RS選股(ETF)' + '.xlsx', encoding='utf-8-sig')


    IDs = apexstock.index.values
    day2 = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+365))) , '%Y-%m-%d' )
    for i, ID in enumerate(IDs):
        ID = str(int(ID))
        try:
            stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
        except:
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
            except Exception as e:
                print(ID)
                print(e)
        f2 = 1
        try:
            if f2 == 1:
                rocp_df = stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP']
                # rocp_df = rocp_df.loc[(rocp_df.index<day2)]
                f2 = 0
            else:
                rocp = pd.Series(stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP'], name = str(ID))
                rocp_df = pd.concat([rocp, rocp_df], axis=1)
        except Exception as e:
            print(e)
            pass

    roc_df = rocp_df/100
    roc_df = roc_df+1   
    season1_profit = abs(roc_df.iloc[:42]).prod()
    season2_profit = abs(roc_df.iloc[42:84]).prod()
    season3_profit = abs(roc_df.iloc[84:126]).prod()
    season4_profit = abs(roc_df.iloc[126:168]).prod()
    season5_profit = abs(roc_df.iloc[168:210]).prod()
    season6_profit = abs(roc_df.iloc[210:]).prod()
    all_profit = abs(roc_df.iloc[:]).prod()
    season1_profit = season1_profit-1
    season2_profit = season2_profit-1
    season3_profit = season3_profit-1
    season4_profit = season4_profit-1
    season5_profit = season5_profit-1
    season6_profit = season6_profit-1
    all_profit = all_profit-1
    season1_profit = season1_profit.sum()
    season2_profit = season2_profit.sum()
    season3_profit = season3_profit.sum()
    season4_profit = season4_profit.sum()
    season5_profit = season5_profit.sum()
    season6_profit = season6_profit.sum()
    all_profit = all_profit.sum()
    profit = {'season1_profit':season1_profit,
            'season2_profit':season2_profit,
            'season3_profit':season3_profit,
            'season4_profit':season4_profit,
            'season5_profit':season3_profit,
            'season6_profit':season4_profit,
            'all_profit':all_profit,
            'number of stock':len(apexstock.index.values)}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([profit_df, all_df])
    display(all_df)
    # print(season1_profit.sum())
    # print(season2_profit.sum())
    # print(season3_profit.sum())
    # print(season4_profit.sum())
    # print(all_profit.sum())

In [3]:
avg_profit_df5 = pd.DataFrame([], columns=all_df.columns)
for ind in all_df.index.values:
    avg_profit_df5.loc[ind] = 100*all_df.loc[ind]/all_df.loc[ind, 'number of stock']
avg_profit_df5.loc['total'] = avg_profit_df5.sum()
display(avg_profit_df5)

,season1_profit,number of stock
2023-06-26 00:00:00,0.943599,100.0
2023-04-24 00:00:00,13.470190,100.0
2023-02-20 00:00:00,14.257476,100.0
2022-12-20 00:00:00,17.146406,100.0
2022-10-19 00:00:00,11.306384,100.0
...,...,...
2010-05-31 00:00:00,5.266460,100.0
2010-03-29 00:00:00,-7.508091,100.0
2010-01-25 00:00:00,13.862181,100.0
2009-11-24 00:00:00,13.493964,100.0


In [3]:
avg_profit_df5.to_excel(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\ETF test\T8.xlsx')

In [1]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-5002, -30, 62)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    f = 1
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if f == 1:
                allstock = stockdata
                f = 0
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            # print(e)
            pass
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            allstock.loc[id, 'RS 50rate>80'] = (allstock.loc[id, 'RS 50rate'] > 80)
            allstock.loc[id, 'RS MA250rate>85'] = (allstock.loc[id, 'RS 250rate']>80)
            allstock.loc[id, 'RS MA20rate>85'] = (allstock.loc[id, 'RS 20rate'] > 80)
            allstock.loc[id, 'RS EMA250rate>70'] = (allstock.loc[id, 'RS EMA250rate']>70)
            allstock.loc[id, 'RS EMA20rate>75'] = (allstock.loc[id, 'RS EMA20rate'] > 75)
            allstock.loc[id, 'ATR250/price'] = (allstock.loc[id, 'ATR250'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR50/price'] = (allstock.loc[id, 'ATR50'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR20/price'] = (allstock.loc[id, 'ATR20'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR250/price<0.03'] = (allstock.loc[id, 'ATR250/price'] < 0.03)
            allstock.loc[id, 'ATR250/price<0.5'] = (allstock.loc[id, 'ATR250/price'] < 0.15)
            allstock.loc[id, 'ATR50/price<0.03'] = (allstock.loc[id, 'ATR50/price'] < 0.03)
            allstock.loc[id, 'ATR20/price<0.03'] = (allstock.loc[id, 'ATR20/price'] < 0.03)            
            allstock.loc[id, 'RS 20rate>70'] = (allstock.loc[id, 'RS 20rate'] > 70)
            allstock.loc[id, 'RS 250rate>70'] = (allstock.loc[id, 'RS 250rate'] > 70)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 250*1000)
            allstock.loc[id, 'Volume 50MA>1000k'] = (allstock.loc[id, 'Volume 50MA'] > 1000*1000)
            allstock.loc[id, 'Mark Tamplate'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
            allstock.loc[id, 'T1'] = all(allstock.loc[id, ['RS MA250rate>85','RS MA20rate>85', 'Volume 50MA>100k']])
            allstock.loc[id, 'T2'] = all(allstock.loc[id, ['ATR250/price<0.5', 'Volume 50MA>1000k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 第一個日期選出符合條件的股票並匯出excel
    # print(allstock['Mark Tamplate'])
    allstock.dropna(subset='Mark Tamplate', inplace=True)
    template1_df = allstock.loc[allstock['T1'], ['產業別', 'T1', 'T2', 'ATR250/price', 'ATR50/price', 'ATR20/price'
                                                 , 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                                 , 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA'
                                                 ,'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    template2_df = allstock.loc[allstock['T2'], ['產業別', 'T1', 'T2', 'ATR250/price', 'ATR50/price', 'ATR20/price'
                                                 , 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                                 , 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA'
                                                 ,'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    template1_df.dropna(inplace=True)
    apexstock = template1_df.sort_values(by='RS EMA250rate', ascending=True).iloc[-50:]
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']].astype(float).round(1)
    apexstock = apexstock.dropna()
    print(f'apexstock len:{len(apexstock.index.values)}')
    print(str(day))
    
    # template1_df.dropna(inplace=True)
    # template2_df.dropna(inplace=True)
    # template1_df = template1_df.sort_values(by='RS MA250rate', ascending=True)
    # template2_num = int(len(template1_df.index.values)/4)
    # template2_df = template2_df.sort_values(by='RS MA250rate', ascending=True)
    # print(f'T2 len:{len(template2_df.index.values)}')
    # template2_df = template2_df.loc[~template2_df.index.isin(template1_df.index)].iloc[-template2_num:]
    # apexstock = pd.concat([template1_df, template2_df])
    # apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']].astype(float).round(1)
    # apexstock = apexstock.dropna()
    # print(f'T1 len:{len(template1_df.index.values)}')
    # print(f'T2 len:{len(template2_df.index.values)}')
    print(f'apexstock len:{len(apexstock.index.values)}')
    print(str(day))
    

    profit_list = []
    IDs = apexstock.index.values
    day2 = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+62))) , '%Y-%m-%d' )
    for i, ID in enumerate(IDs):
        ID = str(int(ID))
        try:
            stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
        except:
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
            except Exception as e:
                print(ID)
                print(e)
                try:
                    apexstock.drop(ID, inplace=True)
                except:
                    pass
                continue
        try:
            initial_price = stock.loc[stock.index>day, 'Adj Close'].iloc[0]
            final_price = stock.loc[stock.index<day2, 'Adj Close'].iloc[-1]
            profit_list.append([ID, (final_price-initial_price)/initial_price])
        except Exception as e:
            print(e)
            try:
                apexstock.drop(ID, inplace=True)
            except:
                pass
            continue

    print(len(apexstock.index.values))
    print(np.shape(profit_list))
    profit_df = pd.DataFrame(profit_list, columns=['ID', 'profit'])
    profit_df.set_index('ID', inplace=True)
    apexstock.loc[apexstock.index.values, 'profit'] = profit_df['profit'].values
    apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/ETF test/stock choose T9/' + str(day).split(' ')[0] + 'RS選股(T9)' + '.xlsx', encoding='utf-8-sig')
    profit = {'season1_profit':profit_df.sum().values,
            'number of stock':len(apexstock.index.values)}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        day_profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([day_profit_df, all_df])
    display(all_df)
    # print(season1_profit.sum())
    # print(season2_profit.sum())
    # print(season3_profit.sum())
    # print(season4_profit.sum())
    # print(all_profit.sum())

     有價證券名稱 市場別    產業別 公開發行/上市(櫃)/發行日
ID                                   
1101     台泥  上市   水泥工業     1962/02/09
1102     亞泥  上市   水泥工業     1962/06/08
1103     嘉泥  上市   水泥工業     1969/11/14
1104     環泥  上市   水泥工業     1971/02/01
1108     幸福  上市   水泥工業     1990/06/06
...     ...  ..    ...            ...
9949     琉園  上櫃  文化創意業     2003/11/21
9950    萬國通  上櫃   塑膠工業     2004/02/17
9951     皇田  上櫃   電機機械     2003/12/18
9960    邁達康  上櫃   運動休閒     2004/12/06
9962     有益  上櫃   鋼鐵工業     2006/07/10

[1788 rows x 4 columns]
day 2009-11-23 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:50
2009-11-23 00:00:00
apexstock len:50
2009-11-23 00:00:00
50
(50, 2)


,season1_profit,number of stock
2009-11-23,7.338871,50


day 2010-01-25 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:50
2010-01-25 00:00:00
apexstock len:50
2010-01-25 00:00:00
50
(50, 2)


,season1_profit,number of stock
2010-01-25,5.080977,50
2009-11-23,7.338871,50


day 2010-03-29 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:50
2010-03-29 00:00:00
apexstock len:50
2010-03-29 00:00:00
50
(50, 2)


...

,season1_profit,number of stock
2023-02-20,4.412379,50
2022-12-19,6.461624,50
2022-10-18,5.762891,50
2022-08-17,-8.438810,50
2022-06-16,4.497393,50
...,...,...
2010-07-29,0.459267,50
2010-05-28,3.875458,50
2010-03-29,-3.255699,50
2010-01-25,5.080977,50


day 2023-04-24 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:50
2023-04-24 00:00:00
apexstock len:50
2023-04-24 00:00:00
50
(50, 2)


,season1_profit,number of stock
2023-04-24,7.403582,50
2023-02-20,4.412379,50
2022-12-19,6.461624,50
2022-10-18,5.762891,50
2022-08-17,-8.438810,50
...,...,...
2010-07-29,0.459267,50
2010-05-28,3.875458,50
2010-03-29,-3.255699,50
2010-01-25,5.080977,50


day 2023-06-26 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:50
2023-06-26 00:00:00
apexstock len:50
2023-06-26 00:00:00
50
(50, 2)


,season1_profit,number of stock
2023-06-26,-1.700130,50
2023-04-24,7.403582,50
2023-02-20,4.412379,50
2022-12-19,6.461624,50
2022-10-18,5.762891,50
...,...,...
2010-07-29,0.459267,50
2010-05-28,3.875458,50
2010-03-29,-3.255699,50
2010-01-25,5.080977,50


In [4]:
display(all_df)
avg_profit_df5 = pd.DataFrame([])
for ind in all_df.index.values:
    avg_profit_df5.loc[ind, 'season1 profit'] = 100*all_df.loc[ind, 'season1_profit']/all_df.loc[ind, 'number of stock']
    avg_profit_df5.loc[ind, 'numbers of stock'] = all_df.loc[ind, 'number of stock']
avg_profit_df5.loc['total'] = avg_profit_df5.sum()
display(avg_profit_df5)
avg_profit_df5.to_excel(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\ETF test\T9.xlsx')

,season1_profit,number of stock
2023-06-26,-1.700130,50
2023-04-24,7.403582,50
2023-02-20,4.412379,50
2022-12-19,6.461624,50
2022-10-18,5.762891,50
...,...,...
2010-07-29,0.459267,50
2010-05-28,3.875458,50
2010-03-29,-3.255699,50
2010-01-25,5.080977,50


,season1 profit,numbers of stock
2023-06-26 00:00:00,-3.400260,50.0
2023-04-24 00:00:00,14.807163,50.0
2023-02-20 00:00:00,8.824758,50.0
2022-12-19 00:00:00,12.923249,50.0
2022-10-18 00:00:00,11.525782,50.0
...,...,...
2010-05-28 00:00:00,7.750917,50.0
2010-03-29 00:00:00,-6.511397,50.0
2010-01-25 00:00:00,10.161955,50.0
2009-11-23 00:00:00,14.677741,50.0


In [1]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-5002, -30, 62)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    f = 1
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if f == 1:
                allstock = stockdata
                f = 0
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            # print(e)
            pass
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            allstock.loc[id, 'RS 50rate>80'] = (allstock.loc[id, 'RS 50rate'] > 80)
            allstock.loc[id, 'RS MA250rate>85'] = (allstock.loc[id, 'RS 250rate']>80)
            allstock.loc[id, 'RS MA20rate>85'] = (allstock.loc[id, 'RS 20rate'] > 80)
            allstock.loc[id, 'RS EMA20rate<99'] = (allstock.loc[id, 'RS EMA20rate'] < 99)
            allstock.loc[id, 'RS EMA250rate>70'] = (allstock.loc[id, 'RS EMA250rate']>70)
            allstock.loc[id, 'RS EMA20rate>75'] = (allstock.loc[id, 'RS EMA20rate'] > 75)
            allstock.loc[id, 'ATR250/price'] = (allstock.loc[id, 'ATR250'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR50/price'] = (allstock.loc[id, 'ATR50'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR20/price'] = (allstock.loc[id, 'ATR20'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR250/price<0.03'] = (allstock.loc[id, 'ATR250/price'] < 0.03)
            allstock.loc[id, 'ATR250/price<0.5'] = (allstock.loc[id, 'ATR250/price'] < 0.15)
            allstock.loc[id, 'ATR50/price<0.03'] = (allstock.loc[id, 'ATR50/price'] < 0.03)
            allstock.loc[id, 'ATR20/price<0.03'] = (allstock.loc[id, 'ATR20/price'] < 0.03)            
            allstock.loc[id, 'RS 20rate>70'] = (allstock.loc[id, 'RS 20rate'] > 70)
            allstock.loc[id, 'RS 250rate>70'] = (allstock.loc[id, 'RS 250rate'] > 70)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 250*1000)
            allstock.loc[id, 'Volume 50MA>1000k'] = (allstock.loc[id, 'Volume 50MA'] > 1000*1000)
            allstock.loc[id, 'Mark Tamplate'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
            allstock.loc[id, 'T1'] = all(allstock.loc[id, ['RS MA250rate>85','RS MA20rate>85', 'RS EMA20rate<99', 'Volume 50MA>100k']])
            allstock.loc[id, 'T2'] = all(allstock.loc[id, ['ATR250/price<0.5', 'Volume 50MA>1000k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 第一個日期選出符合條件的股票並匯出excel
    # print(allstock['Mark Tamplate'])
    allstock.dropna(subset='Mark Tamplate', inplace=True)
    template1_df = allstock.loc[allstock['T1'], ['產業別', 'T1', 'T2', 'ATR250/price', 'ATR50/price', 'ATR20/price'
                                                 , 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                                 , 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA'
                                                 ,'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    template2_df = allstock.loc[allstock['T2'], ['產業別', 'T1', 'T2', 'ATR250/price', 'ATR50/price', 'ATR20/price'
                                                 , 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                                 , 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA'
                                                 ,'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    template1_df.dropna(inplace=True)
    apexstock = template1_df.sort_values(by='RS EMA250rate', ascending=True).iloc[-50:]
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']].astype(float).round(1)
    apexstock = apexstock.dropna()
    print(f'apexstock len:{len(apexstock.index.values)}')
    print(str(day))
    
    # template1_df.dropna(inplace=True)
    # template2_df.dropna(inplace=True)
    # template1_df = template1_df.sort_values(by='RS MA250rate', ascending=True)
    # template2_num = int(len(template1_df.index.values)/4)
    # template2_df = template2_df.sort_values(by='RS MA250rate', ascending=True)
    # print(f'T2 len:{len(template2_df.index.values)}')
    # template2_df = template2_df.loc[~template2_df.index.isin(template1_df.index)].iloc[-template2_num:]
    # apexstock = pd.concat([template1_df, template2_df])
    # apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']].astype(float).round(1)
    # apexstock = apexstock.dropna()
    # print(f'T1 len:{len(template1_df.index.values)}')
    # print(f'T2 len:{len(template2_df.index.values)}')
    print(f'apexstock len:{len(apexstock.index.values)}')
    print(str(day))
    

    profit_list = []
    IDs = apexstock.index.values
    day2 = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+62))) , '%Y-%m-%d' )
    for i, ID in enumerate(IDs):
        ID = str(int(ID))
        try:
            stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
        except:
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
            except Exception as e:
                print(ID)
                print(e)
                try:
                    apexstock.drop(ID, inplace=True)
                except:
                    pass
                continue
        try:
            initial_price = stock.loc[stock.index>day, 'Adj Close'].iloc[0]
            final_price = stock.loc[stock.index<day2, 'Adj Close'].iloc[-1]
            profit_list.append([ID, (final_price-initial_price)/initial_price])
        except Exception as e:
            print(e)
            try:
                apexstock.drop(ID, inplace=True)
            except:
                pass
            continue

    print(len(apexstock.index.values))
    print(np.shape(profit_list))
    profit_df = pd.DataFrame(profit_list, columns=['ID', 'profit'])
    profit_df.set_index('ID', inplace=True)
    apexstock.loc[apexstock.index.values, 'profit'] = profit_df['profit'].values
    apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/ETF test/stock choose T9-/' + str(day).split(' ')[0] + 'RS選股(T9-)' + '.xlsx', encoding='utf-8-sig')
    profit = {'season1_profit':profit_df.sum().values,
            'number of stock':len(apexstock.index.values)}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        day_profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([day_profit_df, all_df])
    display(all_df)
    # print(season1_profit.sum())
    # print(season2_profit.sum())
    # print(season3_profit.sum())
    # print(season4_profit.sum())
    # print(all_profit.sum())

     有價證券名稱 市場別    產業別 公開發行/上市(櫃)/發行日
ID                                   
1101     台泥  上市   水泥工業     1962/02/09
1102     亞泥  上市   水泥工業     1962/06/08
1103     嘉泥  上市   水泥工業     1969/11/14
1104     環泥  上市   水泥工業     1971/02/01
1108     幸福  上市   水泥工業     1990/06/06
...     ...  ..    ...            ...
9949     琉園  上櫃  文化創意業     2003/11/21
9950    萬國通  上櫃   塑膠工業     2004/02/17
9951     皇田  上櫃   電機機械     2003/12/18
9960    邁達康  上櫃   運動休閒     2004/12/06
9962     有益  上櫃   鋼鐵工業     2006/07/10

[1788 rows x 4 columns]
day 2009-11-24 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:50
2009-11-24 00:00:00
apexstock len:50
2009-11-24 00:00:00
50
(50, 2)


,season1_profit,number of stock
2009-11-24,6.746982,50


day 2010-01-25 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:50
2010-01-25 00:00:00
apexstock len:50
2010-01-25 00:00:00
50
(50, 2)


,season1_profit,number of stock
2010-01-25,6.931090,50
2009-11-24,6.746982,50


day 2010-03-29 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:50
2010-03-29 00:00:00
apexstock len:50
2010-03-29 00:00:00
50
(50, 2)


...

,season1_profit,number of stock
2023-02-20,7.128738,50
2022-12-20,8.573203,50
2022-10-19,5.427064,48
2022-08-18,-7.033159,50
2022-06-17,8.488090,50
...,...,...
2010-07-30,1.459327,50
2010-05-31,2.633230,50
2010-03-29,-3.754046,50
2010-01-25,6.931090,50


day 2023-04-24 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:50
2023-04-24 00:00:00
apexstock len:50
2023-04-24 00:00:00
50
(50, 2)


,season1_profit,number of stock
2023-04-24,6.735095,50
2023-02-20,7.128738,50
2022-12-20,8.573203,50
2022-10-19,5.427064,48
2022-08-18,-7.033159,50
...,...,...
2010-07-30,1.459327,50
2010-05-31,2.633230,50
2010-03-29,-3.754046,50
2010-01-25,6.931090,50


day 2023-06-26 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:50
2023-06-26 00:00:00
apexstock len:50
2023-06-26 00:00:00
50
(50, 2)


,season1_profit,number of stock
2023-06-26,0.471799,50
2023-04-24,6.735095,50
2023-02-20,7.128738,50
2022-12-20,8.573203,50
2022-10-19,5.427064,48
...,...,...
2010-07-30,1.459327,50
2010-05-31,2.633230,50
2010-03-29,-3.754046,50
2010-01-25,6.931090,50


In [2]:
display(all_df)
avg_profit_df6 = pd.DataFrame([])
for ind in all_df.index.values:
    avg_profit_df6.loc[ind, 'season1 profit'] = 100*all_df.loc[ind, 'season1_profit']/all_df.loc[ind, 'number of stock']
    avg_profit_df6.loc[ind, 'numbers of stock'] = all_df.loc[ind, 'number of stock']
avg_profit_df6.loc['total'] = avg_profit_df6.sum()
display(avg_profit_df6)
avg_profit_df6.to_excel(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\ETF test\T9-.xlsx')

,season1_profit,number of stock
2023-06-26,0.471799,50
2023-04-24,6.735095,50
2023-02-20,7.128738,50
2022-12-20,8.573203,50
2022-10-19,5.427064,48
...,...,...
2010-07-30,1.459327,50
2010-05-31,2.633230,50
2010-03-29,-3.754046,50
2010-01-25,6.931090,50


,season1 profit,numbers of stock
2023-06-26 00:00:00,0.943599,50.0
2023-04-24 00:00:00,13.470190,50.0
2023-02-20 00:00:00,14.257476,50.0
2022-12-20 00:00:00,17.146406,50.0
2022-10-19 00:00:00,11.306384,48.0
...,...,...
2010-05-31 00:00:00,5.266460,50.0
2010-03-29 00:00:00,-7.508091,50.0
2010-01-25 00:00:00,13.862181,50.0
2009-11-24 00:00:00,13.493964,50.0
